# ERA5-Land Portugal heat/need exposure indices (2010–2012) — native ERA5-Land grid exports

This notebook computes four indices from **ERA5-Land HOURLY** `skin_temperature` over **Portugal (mainland + Azores + Madeira)**:

- **hdd_18**: count of days where **daily mean** skin temperature < 18°C  
- **cdd_25**: count of days where **daily mean** skin temperature > 25°C  
- **extreme_heat**: mean of the **6th–10th hottest** *daily mean* temperatures  
- **extreme_cold**: mean of the **6th–10th coldest** *daily mean* temperatures  

**Key design choice:** computations stay on the **native ERA5-Land grid** (regular lat/lon) throughout, and final exports are also on that same native grid.

Outputs:
- 4 single-band **GeoTIFFs** exported to your **Google Drive** (Earth Engine export tasks).

> Notes  
> - ERA5-Land `skin_temperature` is provided in **Kelvin**; we convert to **°C** as `K - 273.15`.  
> - Indices are computed per pixel, then clipped to Portugal.  
> - Exports run as asynchronous Earth Engine tasks; monitor them in the Earth Engine Tasks UI or via the optional polling cell.


In [1]:
# (Colab) Install Earth Engine Python API (and helpers) if needed
!pip -q install earthengine-api geemap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.8 MB/s eta 0:00:00


In [15]:
import ee
import datetime
import time

# Authenticate & initialize (Colab will prompt you)
ee.Authenticate()
ee.Initialize(project='energy-poverty-from-space')

print("EE initialized:", ee.String('ok').getInfo())

EE initialized: ok


## Parameters

In [16]:
# ---- User parameters ----
DRIVE_FOLDER = "ERA5L_PT_nativegrid_2010_2012"  # Drive folder name to create/use
MAX_PIXELS = 1e13

START_YEAR = 2010
END_YEAR = 2012  # inclusive

# Use DAILY aggregates to avoid hourly explosion.
# (Daily aggregates are already averaged from the hourly ERA5-Land collection.)
ERA5_COLLECTION = "ECMWF/ERA5_LAND/DAILY_AGGR"
TEMP_BAND = "skin_temperature"  # Kelvin (daily mean)

# If you prefer nearest-neighbor behavior (no smoothing), set False.
USE_BILINEAR = False

# --- Load collection (server-side object) ---
era5 = ee.ImageCollection(ERA5_COLLECTION).select(TEMP_BAND)

# --- Native grid definition (ERA5-Land is a regular 0.1° lat/lon grid in EPSG:4326) ---
NATIVE_CRS = "EPSG:4326"
STEP_DEG = 0.1

# Derive an origin aligned to the dataset footprint (for consistent exports).
era5_first = ee.Image(era5.first())
b = era5_first.geometry().bounds()
ring = ee.List(b.coordinates().get(0))
xmin = ee.Number(ee.List(ring.get(0)).get(0))
ymax = ee.Number(ee.List(ring.get(2)).get(1))

# Affine geotransform: [xScale, xShear, xTranslate, yShear, yScale, yTranslate]
NATIVE_TRANSFORM = [STEP_DEG, 0, xmin, 0, -STEP_DEG, ymax]

print("ERA5 collection:", ERA5_COLLECTION)
print("Native CRS:", NATIVE_CRS)
print("Native transform:", NATIVE_TRANSFORM)

ERA5 collection: ECMWF/ERA5_LAND/DAILY_AGGR
Native CRS: EPSG:4326
Native transform: [0.1, 0, <ee.ee_number.Number object at 0x781a058776e0>, 0, -0.1, <ee.ee_number.Number object at 0x781a05877860>]


## Study region: Portugal (incl. Azores + Madeira)

In [17]:
# Portugal boundary from a built-in dataset (includes Atlantic islands)
gaul0 = ee.FeatureCollection("FAO/GAUL/2015/level0")
pt_geom = gaul0.filter(ee.Filter.eq("ADM0_NAME", "Portugal")).geometry()

# Keep a modest buffer (optional) and simplify to avoid very complex polygons in exports.
pt_region = pt_geom.buffer(20000).simplify(1000)  # meters

print("Portugal geometry loaded.")
print("Approx area (km^2):", pt_geom.area().divide(1e6).getInfo())


Portugal geometry loaded.
Approx area (km^2): 88662.99447236159


## Determine ERA5-Land native projection (CRS + transform)

In [18]:
# Inspect the dataset projection / nominal scale (sanity check).
sample_proj = ee.Image(era5.first()).select(TEMP_BAND).projection()

print("Native CRS:", NATIVE_CRS)
print("Native affine transform:", NATIVE_TRANSFORM)
print("Native nominal scale (m):", sample_proj.nominalScale().getInfo())

Native CRS: EPSG:4326
Native affine transform: [0.1, 0, <ee.ee_number.Number object at 0x781a058776e0>, 0, -0.1, <ee.ee_number.Number object at 0x781a05877860>]
Native nominal scale (m): 11131.949079327358


## Build daily mean temperature collection (°C)

In [19]:
# Build a DAILY mean temperature collection (°C) over the requested period.
# NOTE: This uses the ERA5-Land DAILY_AGGR product, so it is *not* iterating over hours.

start_date = ee.Date.fromYMD(START_YEAR, 1, 1)
end_date = ee.Date.fromYMD(END_YEAR + 1, 1, 1)  # exclusive

daily = (
    era5.filterDate(start_date, end_date)
    .map(lambda im: im.subtract(273.15)
         .rename("tmean_c")
         .copyProperties(im, ["system:time_start"]))
    # Clip early to reduce compute for later steps.
    .map(lambda im: im.clip(pt_region))
)

print("Daily images:", daily.size().getInfo())

Daily images: 1096


## Compute yearly indices and average across 2010–2012

In [21]:
# ---- Indices per year (split by metric for speed) ----
# IMPORTANT: If you export only HDD/CDD, you do NOT want Earth Engine to also compute the expensive
# per-pixel sorting used for extremes. So we compute each metric as its own ImageCollection.

def hdd_for_year(year: ee.Number) -> ee.Image:
    y = ee.Number(year)
    y_start = ee.Date.fromYMD(y, 1, 1)
    y_end = y_start.advance(1, "year")
    dly = daily.filterDate(y_start, y_end).select("tmean_c")
    hdd18 = dly.map(lambda im: im.lt(18)).sum().rename("hdd_18")
    return hdd18.set({"year": y})

def cdd_for_year(year: ee.Number) -> ee.Image:
    y = ee.Number(year)
    y_start = ee.Date.fromYMD(y, 1, 1)
    y_end = y_start.advance(1, "year")
    dly = daily.filterDate(y_start, y_end).select("tmean_c")
    cdd25 = dly.map(lambda im: im.gt(25)).sum().rename("cdd_25")
    return cdd25.set({"year": y})

def extremes_for_year(year: ee.Number) -> ee.Image:
    y = ee.Number(year)
    y_start = ee.Date.fromYMD(y, 1, 1)
    y_end = y_start.advance(1, "year")
    dly = daily.filterDate(y_start, y_end).select("tmean_c")

    # Convert the yearly ImageCollection to an array image and sort it per-pixel.
    array = dly.toArray()  # 2D array [image, band]
    image_axis = 0
    band_axis = 1

    # Single-band slice used both as values and keys for sorting.
    temps = array.arraySlice(band_axis, 0, 1)

    # --- Hottest: sort by descending temperature ---
    sorted_desc = temps.arraySort(temps.multiply(-1))
    hottest_6_10 = sorted_desc.arraySlice(image_axis, 5, 10)  # indices 5..9 (6th–10th)
    mean_hot = (
        hottest_6_10
        .arrayReduce(ee.Reducer.mean(), [image_axis])
        .arrayProject([band_axis])
        .arrayFlatten([["extreme_heat"]])
    )

    # --- Coldest: sort by ascending temperature ---
    sorted_asc = temps.arraySort(temps)
    coldest_6_10 = sorted_asc.arraySlice(image_axis, 5, 10)   # indices 5..9 (6th–10th)
    mean_cold = (
        coldest_6_10
        .arrayReduce(ee.Reducer.mean(), [image_axis])
        .arrayProject([band_axis])
        .arrayFlatten([["extreme_cold"]])
    )

    return ee.Image.cat([mean_hot, mean_cold]).set({"year": y})

years = ee.List.sequence(START_YEAR, END_YEAR)

# Build yearly collections separately
hdd_yearly = ee.ImageCollection(years.map(hdd_for_year))
cdd_yearly = ee.ImageCollection(years.map(cdd_for_year))
ext_yearly = ee.ImageCollection(years.map(extremes_for_year))

# Average across years (same intent as before)
avg_hdd18 = hdd_yearly.mean().clip(pt_region)
avg_cdd25 = cdd_yearly.mean().clip(pt_region)
avg_extremes = ext_yearly.mean().clip(pt_region)

# Optional resampling (only applied at the end)
if USE_BILINEAR:
    avg_hdd18 = avg_hdd18.resample("bilinear")
    avg_cdd25 = avg_cdd25.resample("bilinear")
    avg_extremes = avg_extremes.resample("bilinear")

# Convenience single image if you need it for visualization:
avg_indices = ee.Image.cat([avg_hdd18, avg_cdd25, avg_extremes])

print("Bands:", avg_indices.bandNames().getInfo())


Bands: ['hdd_18', 'cdd_25', 'extreme_heat', 'extreme_cold']


## Export GeoTIFFs to Google Drive (native ERA5-Land grid)

In [22]:
def export_band(band_name: str):
    file_base = f"PT_{band_name}_ERA5L_skinT_dailyMean_{START_YEAR}_{END_YEAR}_nativegrid"

    # Export the light-weight images when possible:
    if band_name == "hdd_18":
        image = avg_hdd18
    elif band_name == "cdd_25":
        image = avg_cdd25
    elif band_name in ["extreme_heat", "extreme_cold"]:
        image = avg_extremes.select([band_name])
    else:
        raise ValueError("Unknown band: " + band_name)

    # Using a rectangular region greatly reduces geometry complexity in exports.
    # The raster is already clipped/masked to Portugal, so this does not change the data values.
    export_region = pt_region.bounds()

    task = ee.batch.Export.image.toDrive(
        image=image.toFloat(),
        description=f"EXPORT_{file_base}",
        folder=DRIVE_FOLDER,
        fileNamePrefix=file_base,
        region=export_region,
        crs=NATIVE_CRS,
        crsTransform=NATIVE_TRANSFORM,  # ensures native grid alignment
        maxPixels=MAX_PIXELS,
        fileFormat="GeoTIFF"
    )
    task.start()
    return task

bands = ["hdd_18", "cdd_25", "extreme_heat", "extreme_cold"]
tasks = {b: export_band(b) for b in bands}

print("Started export tasks:")
for b, t in tasks.items():
    print(f"  {b:13s}  task_id={t.id}")


Started export tasks:
  hdd_18         task_id=PWCX4GPKAGK6FLXNNNTT2L2U
  cdd_25         task_id=LWGQ7JX5NDHMIWM4JSN7UGQC
  extreme_heat   task_id=KIXEFXDKIDYEONBDR535DNZP
  extreme_cold   task_id=XLO7XC3Q2CS4CNVSGX26QR5G


## (Optional) Poll task statuses

In [23]:
def show_status():
    for b, t in tasks.items():
        s = t.status()
        print(f"{b:13s} | {s.get('state'):10s} | {s.get('description')}")

# Poll a few times (adjust as you like)
for _ in range(6):
    show_status()
    time.sleep(10)


hdd_18        | READY      | EXPORT_PT_hdd_18_ERA5L_skinT_dailyMean_2010_2012_nativegrid
cdd_25        | READY      | EXPORT_PT_cdd_25_ERA5L_skinT_dailyMean_2010_2012_nativegrid
extreme_heat  | READY      | EXPORT_PT_extreme_heat_ERA5L_skinT_dailyMean_2010_2012_nativegrid
extreme_cold  | READY      | EXPORT_PT_extreme_cold_ERA5L_skinT_dailyMean_2010_2012_nativegrid
hdd_18        | READY      | EXPORT_PT_hdd_18_ERA5L_skinT_dailyMean_2010_2012_nativegrid
cdd_25        | READY      | EXPORT_PT_cdd_25_ERA5L_skinT_dailyMean_2010_2012_nativegrid
extreme_heat  | READY      | EXPORT_PT_extreme_heat_ERA5L_skinT_dailyMean_2010_2012_nativegrid
extreme_cold  | READY      | EXPORT_PT_extreme_cold_ERA5L_skinT_dailyMean_2010_2012_nativegrid
hdd_18        | READY      | EXPORT_PT_hdd_18_ERA5L_skinT_dailyMean_2010_2012_nativegrid
cdd_25        | READY      | EXPORT_PT_cdd_25_ERA5L_skinT_dailyMean_2010_2012_nativegrid
extreme_heat  | READY      | EXPORT_PT_extreme_heat_ERA5L_skinT_dailyMean_2010_2012_na